In [1]:
import geemap, ee

In [3]:
# ee.Authenticate()
# Initialize the library.
ee.Initialize()

In [4]:
%load_ext autoreload
%autoreload 2
import sys  
sys.path.insert(0, '../lib/')



In [46]:
%reload_ext autoreload
import quality_mosaic as qm

# Choose date and location

## Define parameters

In [7]:
date = '2020-06-18'
lat, lon = 17.973859475,-65.988746993


In [8]:
lat, lon = 48.1496822,-123.5552726
date = '2020-10-15'

In [9]:
lat, lon = lat, lon = 33.869066751,-118.405718865
date = '2021-01-18'

## Define Hyperparameters

In [45]:
#define the total number of days before and after the date of interest you want to search``
max_search_window_months = 6
# define the number of days to median filtered S1 images
s1_median_days = 6

Convert to ee objects

In [11]:
point = ee.Geometry.Point([lon, lat])
sample = point.buffer(10).bounds()
roi = point.buffer(100).bounds()

In [44]:
visParamsRGB = {"min": 0, "max": 4000, "bands": ["B4", "B3", "B2"]}
visParamsVV  = {"min": -25, "max": 0, "bands": ["VV"]}

# Get mosaics for S1 and S2

In [47]:
s1_s2 = qm.get_s1_s2(roi=roi, date=date, max_search_window_months=6,s1_median_days=s1_median_days)

Search window from 2020-07-18 to 2021-07-18


In [49]:
keep_bands = ["B2","B3","B4","B8","B8A","B11","B12","VV","VH","FScore"]
s1_s2 = s1_s2.select(keep_bands)

# Show interactive map

Add a marker manually if you want to change the location of the sample pointm

In [50]:
Map = geemap.Map()
Map.add_basemap('SATELLITE')
Map.addLayer(s1_s2, visParamsRGB, 'qm_s2')
Map.addLayer(s1_s2, visParamsVV, 's1')

Map.addLayer(sample,
             {'color':'yellow', 'opacity':0.3},
             name='buffer')

Map.add_marker([lat, lon],tooltip='sample', name='sample', draggable=False)
Map.centerObject(sample, 20)

Map

Map(center=[33.869066767472795, -118.40571869467622], controls=(WidgetControl(options=['position', 'transparen…

In [33]:
if (Map.draw_last_feature is not None):
    
    Map.draw_last_feature.geometry().getInfo()['coordinates']
    new_sample = Map.draw_last_feature.buffer(10).bounds()
    Map.addLayer(new_sample,
             {'color':'green', 'opacity':0.3},
             name='new_buffer')
# Map.draw_last_feature#.geometry().getInfo()['coordinates']

## TODOs:
1. ~~Add S-1 imagery~~
2. ~~Remove all unnecesary bands~~
3. Add mTGSI and BSI indices
4. Try the reduceRegion to capture info within sample buffer